In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

"""
Seguimiento de loss, accuracy y imprimir graficos.
"""
import matplotlib.pyplot as plt
arr_loss_train = []
arr_loss_test = []
arr_acc_train = []
arr_acc_test = []
def loss_plot(epochs, loss):
    plt.plot([*range(epochs)], loss)
    plt.ylabel("Loss")
    plt.xlabel("epochs")
def acc_plot(epochs,acc):
    plt.plot([*range(epochs)], acc)
    plt.ylabel("Accuracy")
    plt.xlabel("epochs")

In [ ]:
"""
Algoritmo de entrenamiento y de validacion de accuraccy.
"""
def train_loop(dataloader, model, loss_fn, optimizer):
    print_every_x_batches = 100
    size = len(dataloader.dataset)
    correct = 0
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # calculate accuracy
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        # print loss during training, not recorded.
        if batch % print_every_x_batches == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    arr_loss_train.append(loss.cpu().detach().numpy().item())
    correct /= size
    arr_acc_train.append(correct)

    


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    model.eval() # pone a la red en moodo evaluacion, desactiva capas dropout.
    with torch.no_grad(): # desactiva el proceso de calculo y guardado de valores intermedios
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    arr_loss_test.append(test_loss)
    arr_acc_test.append(correct)

In [ ]:
"""
Instanciar modelo a ajustar hiperparametros.
"""
from modelo_convolucional import RedConvolucional
model = RedConvolucional()

"""
Definir funcion de costo y optimizador
Explicacion sobre la optimizacion 
https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html
"""
import torch
from torch import nn
# CrossEntropyLoss requiere el output "logits", no es necesario pasarlo por el softmax, ya que lo calcula dentro. https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
loss_function = nn.CrossEntropyLoss()
# se necesita enviar los parametros del modelo al optimizador para que los pueda actualizar.
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
"""
importar funciones utilidad.
"""
from cargar_datos import cargar_datasets, cargar_dataloaders, classes

In [ ]:
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler


def main(num_samples=10, max_num_epochs=10, gpus_per_trial=0.5):
    data_dir = os.path.abspath("./data")
    load_data(data_dir)
    """
    variable de configuracion, Ray tune modificara sus valores en el proceso de ajuste de hiperparametros.
    """
    config = {
        "batch_size": tune.choice([8, 16, 32]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        # red convolucional
        "cant_filtros_conv1": tune.choice([6, 12, 18]),
        "kernel_size_maxpool1": tune.choice([2, 3]),
        "cant_filtros_conv2": tune.choice([16, 22, 28]),
        "kernel_size_maxpool2": tune.choice([2, 3]),
        "full_l1": tune.choice([120, 140, 160]),
        "full_l2": tune.choice([84, 104, 124])
    }

    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    result = tune.run(
        partial(train_cifar, data_dir=data_dir),
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

    best_trained_model = Net(best_trial.config["l1"], best_trial.config["l2"])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_acc = test_accuracy(best_trained_model, device)
    print("Best trial test set accuracy: {}".format(test_acc))


if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    main(num_samples=10, max_num_epochs=10, gpus_per_trial=0)
